In [1]:
from llama_index.legacy import (
    KeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    download_loader,
    load_index_from_storage,
    VectorStoreIndex)

from llama_index.legacy.llms.vertex import Vertex
from langchain.chat_models import ChatVertexAI
from llama_index.legacy.llms.langchain import LangChainLLM
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.legacy.embeddings import LangchainEmbedding
from llama_index.legacy import Document
from llama_index.legacy.node_parser import SimpleNodeParser, HierarchicalNodeParser



from llama_index.legacy.vector_stores.chroma import ChromaVectorStore
import chromadb


from google.oauth2 import service_account
from llama_index.legacy import set_global_service_context
from google.cloud import aiplatform
import json 

imports from og llama index file

In [2]:
from llama_index.legacy import VectorStoreIndex, SimpleDirectoryReader
from llama_index.legacy.embeddings import LangchainEmbedding
from llama_index.legacy.text_splitter import TokenTextSplitter
from langchain_google_vertexai import VertexAIEmbeddings
from llama_index.legacy.llms.vertex import Vertex
from llama_index.legacy.node_parser import SimpleNodeParser, HierarchicalNodeParser
from llama_index.legacy import (
    KeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    download_loader,
    load_index_from_storage,
    VectorStoreIndex)
from llama_index.legacy.retrievers import VectorIndexRetriever
from llama_index.legacy.prompts import (
    ChatMessage,
    ChatPromptTemplate,
    MessageRole,
    PromptTemplate,
)

from llama_index.legacy.postprocessor import NERPIINodePostprocessor, SentenceEmbeddingOptimizer
from llama_index.legacy import ServiceContext
from llama_index.legacy.schema import QueryBundle
from llama_index.legacy.schema import NodeWithScore, TextNode

from pathlib import Path
from tqdm.notebook import tqdm
from google.oauth2 import service_account

from llama_index.legacy import set_global_service_context
import re
import uuid
import os
from pathlib import Path
from pprint import pprint
import pandas as pd
import csv
from typing import List, Tuple, Dict
import time

from llama_index.legacy.vector_stores.chroma import ChromaVectorStore
from llama_index.legacy import StorageContext
from llama_index.legacy.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
from llama_index.core import Settings

from llama_index.legacy.llms.langchain import LangChainLLM
from langchain.chat_models import ChatVertexAI
from llama_index.legacy import Response
from llama_index.legacy.response_synthesizers import Refine
from llama_index.legacy.evaluation import SemanticSimilarityEvaluator
from llama_index.legacy.evaluation import RelevancyEvaluator
from llama_index.legacy.embeddings import SimilarityMode

import nest_asyncio
nest_asyncio.apply()

# additional
from llama_index.legacy.postprocessor import LongContextReorder
from llama_index.legacy.retrievers import AutoMergingRetriever
from llama_index.legacy.query_engine import TransformQueryEngine
from llama_index.legacy.query_engine import RetrieverQueryEngine
from llama_index.legacy.indices.query.query_transform.base import (
    HyDEQueryTransform,StepDecomposeQueryTransform
)
from llama_index.legacy.node_parser import get_leaf_nodes

# llama_debug = LlamaDebugHandler(print_trace_on_end=True)
# callback_manager = CallbackManager([llama_debug])

In [3]:
credentials = service_account.Credentials.from_service_account_file("credentials\\vertex-test-417403-ce72ad032af7.json")
vertex_ai = Vertex(model="text-bison", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2)
chat_vertex_ai = ChatVertexAI(model_name="chat-bison-32k", project=credentials.project_id, location= "asia-southeast1", credentials=credentials, temperature=0.2, max_output_tokens= 8192) # max for bison 32k                                 
embed_model = LangchainEmbedding(VertexAIEmbeddings(model_name='textembedding-gecko-multilingual@latest'))

c:\Users\Aekky\anaconda3\envs\llama_env\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.vertexai.ChatVertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import ChatVertexAI`.
  warn_deprecated(


In [4]:
service_context = ServiceContext.from_defaults(llm=vertex_ai, embed_model=embed_model, chunk_size=1024, chunk_overlap=20)
set_global_service_context(service_context)
# need to set global service context becuase if not index = VectorStoreIndex.from_vector_store(vector_store, embed_model=Settings.embed_model) will produce an error
# it will say that it requires OPENAI key etc
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=3900, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='textembedding-gecko-multilingual@latest', embed_batch_size=10, callback_manager=<llama_index.legacy.callbacks.base.CallbackManager object at 0x0000022808433F90>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.legacy.callbacks.base.CallbackManager object at 0x0000022808433F90>, id_func=<function default_id_func at 0x00000228784DD300>, chunk_size=1024, chunk_overlap=20, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.legacy.logger.base.LlamaLogger object at 0x00000228544F4A50>, callback_manager=<llama_index.legacy.callbacks.base.

Inspecting json data

In [5]:
json_file = "baby_0_3.json"
with open(json_file, 'r') as f:
    json_data = json.load(f)
    
# print("the number of documents: " , len(json_data))
# for i in json_data:
#     print(i) # str

How to create documents from a list of strings: 
https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_documents.html#defining-documents

In [6]:
docs = [Document(text=t) for t in json_data]

Create local ChromaDB Vectore Store

In [7]:
# # Create index and persist to Chroma vector store

# db = chromadb.PersistentClient(path="chroma_db")
# chroma_collection = db.get_or_create_collection("chroma_db")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)

# index = VectorStoreIndex.from_documents(
#     documents = docs, storage_context=storage_context, embed_model=embed_model
# )

In [8]:
# Load index from persisted vector store

db = chromadb.PersistentClient(path="chroma_db")
chroma_collection = db.get_or_create_collection("chroma_db")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

Hierarchical Node Parsing

In [9]:
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[1024, 512, 256]
)
nodes = node_parser.get_nodes_from_documents(docs)

In [10]:
leaf_nodes = get_leaf_nodes(nodes)
nodes_by_id = {node.node_id: node for node in nodes}

In [11]:
auto_merging_context = ServiceContext.from_defaults(
    llm=LangChainLLM(ChatVertexAI(model_name="chat-bison@002",
                                         temperature=0.2,
                                         max_output_tokens=256)
                                         ),
    embed_model= LangchainEmbedding(VertexAIEmbeddings(model_name='textembedding-gecko-multilingual@latest')),
    node_parser=node_parser,
)

In [12]:
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

automerging_index = VectorStoreIndex(
    leaf_nodes, storage_context=storage_context, service_context=auto_merging_context
)

automerging_index.storage_context.persist(persist_dir="./merging_index")

In [13]:
automerging_retriever = automerging_index.as_retriever(
    similarity_top_k=10
)

retriever = AutoMergingRetriever(
    automerging_retriever, 
    automerging_index.storage_context, 
    verbose=True
)

Query engine

In [14]:
reorder = LongContextReorder()
hyde = HyDEQueryTransform(llm=vertex_ai, include_original=True)
query_engine = RetrieverQueryEngine.from_args(retriever,
                                              node_postprocessors=[reorder],
                                              )
query_engine = TransformQueryEngine(query_engine, query_transform=hyde)


Response production

In [15]:
query = "ถ้ามีอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกายควรปรึกษาแพทย์ด้านใด"
response=query_engine.query(query)

> Merging 2 nodes into parent node.
> Parent node id: 34c2ef2e-51e8-477e-afdc-c51e5eec3ba5.
> Parent node text: การตอบสนองของพ่อแม่เมื่อเกิดอาการ โดยการตามใจ ทำให้ลูกแสดงอาการมากขึ้น วิธีการแก้ไข เด็กปกติที่กล...



Inspecting response and context nodes

In [16]:
response.response

' ข้อมูลที่ให้มาไม่มีข้อมูลเกี่ยวกับอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกาย ดังนั้นจึงไม่สามารถตอบคำถามได้'

In [17]:
print(response.source_nodes[0])

Node ID: b1384542-fde3-4c50-814f-4e77cdf40f1b
Text: โรคบางอย่างท่ีนำมาด้วยอาการท้องผูก
เช่นโรคที่มีไทรอยด์ฮอร์โมนต�่า เด็กมักมีปัญหา พัฒนาการช้าร่วมด้วย ฯลฯ
หากสงสัยโรคดังกล่าวควรพาลูกไปพบแพทย์ วิธีการแก้ไข 1.
Score:  0.744



In [18]:
response.source_nodes[0].node.text

'โรคบางอย่างท่ีนำมาด้วยอาการท้องผูก เช่นโรคที่มีไทรอยด์ฮอร์โมนต�่า เด็กมักมีปัญหา พัฒนาการช้าร่วมด้วย ฯลฯ หากสงสัยโรคดังกล่าวควรพาลูกไปพบแพทย์ วิธีการแก้ไข 1.'

In [19]:
len(response.source_nodes)

9

Context Transformations - PII Example
- adding context to model response generation

In [20]:
pii_processor = NERPIINodePostprocessor()

def filter_pii_fn(**kwargs):
    # run optimizer
    query_bundle = QueryBundle(query_str=kwargs["query_str"])

    new_nodes = pii_processor.postprocess_nodes(
        [NodeWithScore(node=TextNode(text=kwargs["context_str"]))],
        query_bundle=query_bundle,
    )
    new_node = new_nodes[0]
    return new_node.get_content()

In [21]:
vector_retriever = index.as_retriever(similarity_top_k=2)

In [22]:
def produce_response(query):
    response = query_engine.query(query)
    return response

def produce_context(query):
    retrieved_nodes = vector_retriever.retrieve(query)
    context_str = "\n\n".join([n.get_content() for n in retrieved_nodes])
    return context_str

# model_resp_df['model_response'] = model_resp_df['question'].apply(produce_response)
# time.sleep(5)
# model_resp_df['context'] = model_resp_df['question'].apply(produce_context)

In [23]:
query = "ถ้ามีอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกายควรปรึกษาแพทย์ด้านใด"
response = produce_response(query)
context =  produce_context(query)

print("response:" , response)
print("context: " , context)

> Merging 2 nodes into parent node.
> Parent node id: 34c2ef2e-51e8-477e-afdc-c51e5eec3ba5.
> Parent node text: การตอบสนองของพ่อแม่เมื่อเกิดอาการ โดยการตามใจ ทำให้ลูกแสดงอาการมากขึ้น วิธีการแก้ไข เด็กปกติที่กล...

response:  ข้อมูลที่ให้มาไม่มีข้อมูลเกี่ยวกับอาการเจ็บหน้าอกและหอบหืดขณะออกกำลังกาย ดังนั้นจึงไม่สามารถตอบคำถามได้
context:  โรคบางอย่างท่ีนำมาด้วยอาการท้องผูก เช่นโรคที่มีไทรอยด์ฮอร์โมนต�่า เด็กมักมีปัญหา พัฒนาการช้าร่วมด้วย ฯลฯ หากสงสัยโรคดังกล่าวควรพาลูกไปพบแพทย์ วิธีการแก้ไข 1. ถ้าลูกอายุ 1-3 เดือน ยังกินนมแม่และนมผสม ควรงดนมผสมทั้งหมด ถ้าไม่ดีขึ้นให้น�้าส้มคั้น โดยไม่ต้องเจือจาง ถ้าอุจจาระลูกยังแข็ง ให้เปลี่ยนเป็นน�้าลูกพรุน 2. แนะนำให้ลูกกินอาหารที่มีกากใยมากขึ้น เช่น ผัก ผลไม้ ธัญพืช และดื่มน�้ามากๆ 3. ฝึกนัง่อุจจาระทกุวนัหลงัอาหารเช้า เยน็ นานคร้ังละ 5-10 นาที พ่อแม่ควรทำบนัทึกไว้ทกุครัง้ พร้อมทั้งให้คำชมเชยเมื่อลูกทำได้ หากลูกต้านการใช้กระโถน ควรเว้นไปสักระยะก่อน รอจนกว่าลูกจะ พร้อม แล้วค่อยฝึกใหม่ 4. การใช้ยาระบายหรือยาสวนควรอยู่ภายใต้การดูแลของแพทย์ ปัญหาที่พบบ่อยใ